# Homework 7

## FINM 36700 - 2023

### UChicago Financial Mathematics

**Professor**
* Mark Hendricks
* hendricks@uchicago.edu

**Students**

Anya Zakharov azakharov@uchicago.edu

Gleb Skvortsov goskvortsov@uchicago.edu

Kai Wen Tay kaiwent@uchicago.edu

Ka Hang Toong khtoong@uchicago.edu


Tikhonov Sergei tikhonov@uchicago.edu


## Modules

In [1]:
import os
import pandas as pd
from scipy.stats import norm
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from arch import arch_model
from arch.univariate import GARCH
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

import matplotlib.pyplot as plt

## Helping functions

In [2]:
def performance_summary(return_data):

    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*12)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(12))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']
    
    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    summary_stats['Min'] = return_data.min()
    summary_stats['Max'] = return_data.max()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [3]:
def regression_based_performance(factor,fund_ret,rf,constant = True):

    if constant:
        X = sm.tools.add_constant(factor)
    else:
        X = factor
    y=fund_ret
    model = sm.OLS(y,X,missing='drop').fit()
    
    if constant:
        beta = model.params[1:]
        alpha = round(float(model.params['const']),6) *12

        
    else:
        beta = model.params
    treynor_ratio = ((fund_ret - rf).mean()*12)/beta[0]
    tracking_error = (model.resid.std()*np.sqrt(12))
    if constant:        
        information_ratio = model.params[0]*12/tracking_error
    r_squared = model.rsquared
    if constant:
        return (beta,treynor_ratio,information_ratio,alpha,r_squared,tracking_error,model.resid,model)
    else:
        return (beta,treynor_ratio,r_squared,tracking_error,model.resid)

## Reading Data

In [4]:
gmo_total_ret = pd.read_excel('gmo_analysis_data.xlsx', sheet_name = 'returns (total)', index_col = 0)
gmo_total_ret.index.name = 'Date'

In [5]:
rf = pd.read_excel('gmo_analysis_data.xlsx', sheet_name = 'risk-free rate', index_col = 0)
rf.index.name = 'Date'

In [6]:
gmo_signals = pd.read_excel('gmo_analysis_data.xlsx', sheet_name = 'signals', index_col = 0)
gmo_signals.index.name = 'Date'

In [7]:
gmo_excess_ret = gmo_total_ret.subtract(rf['US3M'], axis = 0)

## 2) Analyzing GMO

#### This section utilizes data in the file, `gmo_analysis_data.xlsx`.
#### Examine GMO's performance. Use the risk-free rate to convert the total returns to excess returns

### 2.1) Calculate the mean, volatility, and Sharpe ratio for GMWAX. Do this for three samples:

### • from inception through 2011
### • 2012-present
### • inception - present

In [8]:
time_period = [['1993','2011'], ['2012','2023'], ['1993','2023']]
gmwax_performance = []

for period in time_period:
    summary = performance_summary(gmo_excess_ret.loc[period[0]:period[1], ['GMWAX']])
    summary.index = [f'{period[0]}-{period[1]}']
    gmwax_performance.append(summary)

pd.concat(gmwax_performance)[['Mean', 'Volatility', 'Sharpe Ratio']]

,Mean,Volatility,Sharpe Ratio
1993-2011,0.015827,0.125011,0.126603
2012-2023,0.036436,0.094503,0.385556
1993-2023,0.024859,0.112537,0.220898


### Has the mean, vol, and Sharpe changed much since the case?

Since 2012, mean, volatility, and Sharpe ratio have increased dramatically, indicating that GMO's forecasts indeed worked. 

### 2.2 GMO believes a risk premium is compensation for a security's tendency to lose money at "bad times". For all three samples, analyze extreme scenarios by looking at -
### • Min return
### • 5th percentile (VaR-5th)
### • Maximum  Drawdown

In [9]:
time_period = [['1993','2011'], ['2012','2023'], ['1993','2023']]
gmwax_performance = []

for period in time_period:
    summary = performance_summary(gmo_excess_ret.loc[period[0]:period[1], ['GMWAX']])
    summary.index = [f'{period[0]}-{period[1]}']
    gmwax_performance.append(summary)

pd.concat(gmwax_performance)[['Min', 'VaR (0.05)', 'Max Drawdown']]

,Min,VaR (0.05),Max Drawdown
1993-2011,-0.149179,-0.059806,-0.472946
2012-2023,-0.118650,-0.037826,-0.226046
1993-2023,-0.149179,-0.047061,-0.472946


### 2.2.a) Does GMWAX have high or low tail-risk as seen by these stats

GMWAX seems to have low tail-risk as depicted by the tail risk statistics above. 

### 2.2.b) Does that vary much across the two subsamples?

The tail risk is especially low in the latter sub-period of 2012-2023.

### 2.3) For all three samples, regress excess returns of GMWAX on excess returns of SPY.

### 2.3.a) Report the estimated alpha, beta, and r-squared.

In [10]:
time_period = [['1993','2011'], ['2012','2023'], ['1993','2023']]
gmwax_regress = []

for period in time_period: 
    fund_ret = gmo_excess_ret.loc[period[0]:period[1], ['GMWAX']].dropna()
    factor = gmo_excess_ret.loc[fund_ret.index[0]:fund_ret.index[-1], ['SPY']]
    reg = regression_based_performance(factor, fund_ret, 0)
    beta_mkt = reg[0][0]
    alpha = reg[3]
    r_squared = reg[4]
    gmwax_regress.append(pd.DataFrame([[beta_mkt, alpha, r_squared]], 
                                       columns=['SPY Beta', 'Alpha', 'R-Squared'],
                                       index = [f'{period[0]}-{period[1]}']))

reg_performance = pd.concat(gmwax_regress)
reg_performance

,SPY Beta,Alpha,R-Squared
1993-2011,0.539615,-0.005748,0.507129
2012-2023,0.573764,-0.032652,0.754377
1993-2023,0.550609,-0.016572,0.582145


### 2.3.b) Is GMWAX a low-beta strategy? Has that changed since the case?

Type of beta strategy: we can see relatively moderate beta that can indicate this it's indeed low-beta strategy

Stability: The beta is stable in subsamples

### 2.3.c) Does GMWAX provide alpha? Has that changed across the subsamples?

Even though the alpha increases (becomes less negative) during 2012-2022, GMWAX has a negative alpha across both sub-samples.

## 3 Forecast Regressions

#### This section utilizes data in the file,`gmo_analysis_data.xlsx`.

### 3.1) Consider the lagged regression, where the regressor, ($X$), is a period behind the target, ($r^{SPY}$).
\begin{align}
r^{SPY}_t = \alpha^{SPY,X}+(\beta^{SPY,X})'X_{t-1}+\epsilon^{SPY,X}_t
\end{align}
### Estimate (1) and report the $R^2$, as well as the OLS estimates for $\alpha$ and $\beta$. Do this for...
- $X$ as a single regressor, the dividend-price ratio.
- $X$ as a single regressor, the earnings-price ratio.
- $X$ as three regressors, the dividend-price ratio, the earnings-price ratio, and the 10-year yield.

### For each, report the r-squared.

In [11]:
def predict_performance(SPY_data):
    
    fund_returns = SPY_data
    signal_sets = [['DP'], ['EP'], ['DP', 'EP', 'US10Y']]
    forecast_results = []
    
    for set_of_signals in signal_sets:
        
        shifted_factor = gmo_signals[set_of_signals].shift(1)
        regression_result = regression_based_performance(shifted_factor, fund_returns, 0)
        beta_values = []
        header_columns = []
        signal_names = []
        
        for signal in set_of_signals:
            header_columns.append(f'{signal}-Beta')
            signal_names.append(signal)
            
        signal_index = ', '.join(signal_names) if len(set_of_signals) > 1 else set_of_signals[0]
            
        beta_values.extend([regression_result[0][idx] for idx in range(len(set_of_signals))])
        beta_values.extend([regression_result[3], regression_result[4]])

        additional_columns = ['Alpha', 'R-Squared']
        header_columns.extend(additional_columns)
        
        forecast_results.append(pd.DataFrame([beta_values], columns=header_columns, index=[signal_index]))
        
    return forecast_results


In [13]:
forecasts = predict_performance(gmo_total_ret.loc[:, ['SPY']])

In [14]:
forecasts[0]

,DP-Beta,Alpha,R-Squared
DP,0.009516,-0.113772,0.009359


In [15]:
forecasts[1]

,EP-Beta,Alpha,R-Squared
EP,0.003252,-0.073932,0.008692


In [16]:
forecasts[2]

,DP-Beta,EP-Beta,US10Y-Beta,Alpha,R-Squared
"DP, EP, US10Y",0.008023,0.002694,-0.000982,-0.180768,0.016364


### 3.2) For each of the three regressions, let’s try to utilize the resulting forecast in a trading strategy.
- Build the forecasted SPY returns: $\hat{r}^{SPY}_{t+1}$. Note that this denotes the forecast made using $X_t$ to forecast the $(t+1)$ return.
- Set the scale of the investment in SPY equal to 100 times the forecasted value:
$
w_t = 100 \hat{r}^{SPY}_{t+1}
$
- We are not taking this scaling too seriously. We just want the  strategy  to  go  bigger  inperiods where the forecast is high and to withdraw in periods where the forecast is low, or even negative.
- Calcualte the return on this strategy:
$
r^X_{t+1} = w_tr^{SPY}_{t+1}
$

#### You should now have the trading strategy returns, $r^x$ for each of the forecasts. For each strategy, estimate:
- mean, volatility, Sharpe,
- max-drawdown
- market alpha
- market beta
- market Information

In [17]:
dp_forecast_rtn = (gmo_signals.loc[:,'DP'].shift(1).to_frame() * forecasts[0]['DP-Beta'])+forecasts[0]['Alpha']/12
dp_forecast_rtn = dp_forecast_rtn.rename(columns={'DP':'Forecasted Return'}) * 100
dp_strat_rtn = pd.DataFrame(dp_forecast_rtn['Forecasted Return']*gmo_total_ret.loc[:,['SPY']]['SPY'], 
                            columns=dp_forecast_rtn.columns, index=dp_forecast_rtn.index)

In [18]:
ep_forecast_rtn = (gmo_signals.loc[:,'EP'].shift(1).to_frame() * forecasts[1]['EP-Beta'])+forecasts[1]['Alpha']/12
ep_forecast_rtn = ep_forecast_rtn.rename(columns={'EP':'Forecasted Return'}) * 100
ep_strat_rtn = pd.DataFrame(ep_forecast_rtn['Forecasted Return']*gmo_total_ret.loc[:,['SPY']]['SPY'], 
                            columns=ep_forecast_rtn.columns, index=ep_forecast_rtn.index)

In [19]:
forecasted_rets = (np.array(gmo_signals.shift(1).loc[:,['DP','EP','US10Y']]) @ np.array(forecasts[2].loc[:,['DP-Beta','EP-Beta','US10Y-Beta']].T))
fac3_forecast_rtn = (pd.DataFrame(forecasted_rets, columns = ['Forecasted Return'], index= gmo_signals.index)) 
fac3_forecast_rtn['Forecasted Return'] = (fac3_forecast_rtn['Forecasted Return'] + float(forecasts[2]['Alpha']/12))*100
fac3_strat_rtn = pd.DataFrame(fac3_forecast_rtn['Forecasted Return'] *gmo_total_ret.loc[:,['SPY']]['SPY'],
                              columns=fac3_forecast_rtn.columns, index=fac3_forecast_rtn.index)

In [41]:
def summarize_strategies(strategies, factor, rf):

    strat_summary = []
    for name, data in strategies:
        perf_summary = performance_summary(data)
        perf_summary['Negative Risk Premium Months'] = len(data[data['Forecasted Return'] - rf['US3M'] < 0])
        perf_summary['Total Months'] = len(data)
        perf_summary.index = [name]
        reg = regression_based_performance(factor[data.index[0]:], data, 0)
        perf_summary['Market Beta'] = reg[0][0]
        perf_summary['Market Alpha'] = reg[3]
        perf_summary['Market Information Ratio'] = reg[2]
        strat_summary.append(perf_summary)

    strat_summary_df = pd.concat(strat_summary)
    return strat_summary_df

In [43]:
strategies = [
    ('DP', dp_strat_rtn.dropna()),
    ('EP', ep_strat_rtn.dropna()),
    ('DP-EP-US10Y', fac3_strat_rtn.dropna())
]
factor = gmo_excess_ret.loc[:, ['SPY']]
strat_summary_df = summarize_strategies(strategies, factor, rf)
strat_summary_df.loc[:, ['Mean', 'Volatility', 'Sharpe Ratio', 'Max Drawdown',
                                    'Market Beta', 'Market Alpha', 'Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.109539,0.148858,0.735859,-0.656967,0.861746,0.041112,0.549044
EP,0.108055,0.128905,0.838249,-0.385317,0.733554,0.049800,0.732559
DP-EP-US10Y,0.125094,0.145603,0.859145,-0.524606,0.778129,0.063300,0.721235


### 3.3) GMO believes a risk premium is compensation for a security's tendency to lose money at "bad times". Let's consider risk characteristics.

### 3.3.a) For both strategies, the market, and GMO, calculate the monthly VaR for $\pi=.05$. Just use the quantile of the historic data for this VaR calculation.

In [44]:
market_summary = performance_summary(gmo_excess_ret.loc[:,['SPY']])
gmo_summary = performance_summary(gmo_excess_ret.loc[:,['GMWAX']].dropna())
strat_var = pd.concat([strat_summary_df.loc[:,['VaR (0.05)']],
                       market_summary.loc[:,['VaR (0.05)']],
                       gmo_summary.loc[:,['VaR (0.05)']]])
strat_var.T

,DP,EP,DP-EP-US10Y,SPY,GMWAX
VaR (0.05),-0.052335,-0.053892,-0.064082,-0.073525,-0.047061


### 3.3.b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

In [45]:
def summarize_strategies(strategies, start_date, end_date):

    strat_summary = []

    for name, data in strategies:
        strat = data[start_date:end_date]['Forecasted Return'].to_frame('Forecasted Returns')
        perf_summary = performance_summary(strat)  
        perf_summary.index = [name]
        strat_summary.append(perf_summary)

    strat_summary_df = pd.concat(strat_summary)
    return strat_summary_df.loc[:, ['Mean', 'Volatility', 'Sharpe Ratio', 'Max Drawdown']]

In [46]:
strategies = [
    ('DP', dp_strat_rtn.dropna()),
    ('EP', ep_strat_rtn.dropna()),
    ('DP-EP-US10Y', fac3_strat_rtn.dropna()),
    ('Risk Free Rate', rf['US3M'].to_frame('Forecasted Return'))
]

summary_df = summarize_strategies(strategies, '2000', '2011')
summary_df

,Mean,Volatility,Sharpe Ratio,Max Drawdown
DP,0.039709,0.186016,0.213468,-0.656967
EP,0.037709,0.134767,0.279805,-0.385317
DP-EP-US10Y,0.061471,0.158851,0.386972,-0.524606
Risk Free Rate,0.023062,0.005785,3.986632,0.000000


**Answer**: each dynamic portfolio (DP, EP, combined) significantly outperforms risk-free rate over 2000-2011.

### 3.3.c) Based on the regression estimates, in how many periods do we estimate a negative risk premium?

In [47]:
neg_risk_premium = strat_summary_df.loc[:,['Negative Risk Premium Months', 'Total Months']]
T = neg_risk_premium['Total Months']
neg_risk_premium['Negative Risk Premium Months (%)'] = round(neg_risk_premium['Negative Risk Premium Months'] * 100 / T, 2)
neg_risk_premium

,Negative Risk Premium Months,Total Months,Negative Risk Premium Months (%)
DP,139,368,37.77
EP,139,368,37.77
DP-EP-US10Y,138,368,37.50


### 3.3.d) Do you believe the dynamic strategy takes on extra risk?

According to calculations provided above, we can conclude that:

1. The volatilities of the dynamic strategies and SPY are very close to each other
2. The tail risk statistics (VaR 0.05) of the dynamic strategies are lower than that of SPY
3. Other statistics (Max Drawdown) is similar among all the strategies

In [48]:
market_summary.loc[:,['Mean','Volatility','Sharpe Ratio','VaR (0.05)','Max Drawdown']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown
SPY,0.07946,0.149097,0.532939,-0.073525,-0.560012


## 4. Out-of-Sample Forecasting

This section utilizes data in the file, `gmo_analysis_data.xlsx`.

Reconsider the problem above, of estimating (1) for $x$. The reported $R^2$ was the in-sample $R^2$ it examined how well the forecasts fit in the sample from which the parameters were estimated. <br><br>

**In particular, focus on the case of using both dividend-price and earnings-price as signals.**

Let's consider the out-of-sample r-squared. To do so, we need the following:
- Start at $t=60$.
- Estmiate (1) only using data through time $t$.
- Use the estimated parameters of (1), along with $x_{t+1}$ to calculate the out-of-sample forecast for the following period, $t+1$.
\begin{align}
\hat{r}^{SPY}_{t+1} = \hat{a}^{SPY,x}_t+(\beta^{SPY,x})'x_t 
\end{align}
- Calculate the $t+1$ forecast error,
\begin{align}
  e^x_{t+1} = r^{SPY}_{t+1} - \hat{r}^{SPY}_{t+1}
\end{align}
- Move to $t=61$, and loop through the rest of the sample.

You now have the time-series of out-of-sample prediction errors, $e^x$.

Calculate the time-series of out-of-sample prediction errors $e^0$, which are based on the null forecast:
\begin{align*}
\bar{r}^{SPY}_{t+1} &= \frac{1}{t}\sum^{t}_{i=1}r^{SPY}_i \\
e^0_{t+1} &= r^{SPY}_{t+1} - \bar{r}^{SPY}_{t+1}
\end{align*}


In [99]:
def calculate_oos_r_squared(data, factors, start_index):

    target = data['SPY']

    X = sm.add_constant(factors)

    forecast_errors = []
    null_errors = []

    for i in range(start_index, len(data)):
        current_X = X.iloc[:i]
        current_Y = target.iloc[:i]

        model = sm.OLS(current_Y, current_X, missing='drop').fit()
        null_forecast = current_Y.mean()
        prediction = model.predict(X.iloc[i:i+1])
        actual = target.iloc[i]
        forecast_errors.append(prediction - actual)
        null_errors.append(null_forecast - actual)

    RSS = np.sum(np.square(forecast_errors))
    TSS = np.sum(np.square(null_errors))

    oos_r_squared = 1 - (RSS / TSS)

    return oos_r_squared, model

In [101]:
factor = gmo_signals.loc[:,'DP'].shift(1).to_frame()
fund_ret = gmo_total_ret.loc[factor.index[0]:,['SPY']]
reg_dp = calculate_oos_r_squared(fund_ret,factor,60)
OOS_RSquared_dp = reg_dp[0]
OOS_r2_dp = pd.DataFrame([[OOS_RSquared_dp]], columns = ['OOS R-Squared'], index = ['DP'])
reg_dp_params = reg_dp[1]

factor = gmo_signals.loc[:,'EP'].shift(1).to_frame()
fund_ret = gmo_total_ret.loc[factor.index[0]:,['SPY']]
reg_ep = calculate_oos_r_squared(fund_ret,factor,60)
OOS_RSquared_ep = reg_ep[0]
OOS_r2_ep = pd.DataFrame([[OOS_RSquared_ep]], columns = ['OOS R-Squared'], index = ['EP'])
reg_ep_params = reg_ep[1]

factor = gmo_signals.loc[:,['DP','EP']].shift(1)
fund_ret = gmo_total_ret.loc[factor.index[0]:,['SPY']]
reg_epdp = calculate_oos_r_squared(fund_ret,factor,60)
OOS_r2_epdp = reg_epdp[0]
OOS_r2_epdp = pd.DataFrame([[OOS_r2_epdp]], columns = ['OOS R-Squared'], index = ['DP-EP'])
reg_epdp_params = reg_epdp[1]

factor = gmo_signals.loc[:,['DP','EP','US10Y']].shift(1)
fund_ret = gmo_total_ret.loc[factor.index[0]:,['SPY']]
reg_all = calculate_oos_r_squared(fund_ret,factor,60)
OOS_RSquared_all = reg_all[0]
OOS_r2_all = pd.DataFrame([[OOS_RSquared_all]], columns = ['OOS R-Squared'], index = ['All'])
reg_all_params = reg_all[1]

### 4.1) Report the out-of-sample $R^2$:
\begin{align}
 R^2_{OOS} \equiv 1-\frac{\sum^T_{i=61}(e^x_i)^2}{\sum^T_{i=61}(e^0_i)^2} 
\end{align}
### note that unlike an in-sample r-squared, the out-of-sample r-squared can be anywhere between $(-\infty,1]$.

In [102]:
oos_r2_sum = pd.concat([OOS_r2_dp,OOS_r2_ep,OOS_r2_epdp,OOS_r2_all])
oos_r2_sum.T

,DP,EP,DP-EP,All
OOS R-Squared,-0.002074,-0.006394,-0.017227,-0.030651


### Did this forecasting strategy produce a positive OOS r-squared?

All strategies produced negative out-of-sample r-squared. It means that the forecasts of all those models are worse than simple mean.

### 4.2) Re-do problem 3.2 using this OOS forecast. 

In [103]:
def OOS_strat(df, factors, start, weight):
    y = df['SPY']
    X = sm.add_constant(factors)
    returns = []

    for i in range(start, len(df)):
        currX = X.iloc[:i]
        currY = y.iloc[:i]

        reg = sm.OLS(currY, currX, missing='drop').fit()

        pred = reg.predict(X.iloc[[i]])[0]
        w = pred * weight
        returns.append(df.iloc[i]['SPY'] * w)

    df_strat = pd.DataFrame(data=returns, index=df.index[start:], columns=['Strat Returns'])
    return df_strat

In [104]:
factor = gmo_signals.loc[:,'EP'].shift(1).to_frame()
fund_ret= gmo_total_ret.loc[factor.index[0]:,['SPY']]
OOS_EP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'EP_OOS_Returns'})

In [105]:
factor = gmo_signals.loc[:,'DP'].shift(1).to_frame()
fund_ret= gmo_total_ret.loc[factor.index[0]:,['SPY']]
OOS_DP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'DP_OOS_Returns'})

In [106]:
factor = gmo_signals.loc[:,['DP','EP']].shift(1)
fund_ret= gmo_total_ret.loc[factor.index[0]:,['SPY']]
OOS_EPDP_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'DP-EP_OOS_Returns'})

In [107]:
factor = gmo_signals.loc[:,['DP','EP','US10Y']].shift(1)
fund_ret= gmo_total_ret.loc[factor.index[0]:,['SPY']]
OOS_all_predict = OOS_strat(fund_ret,factor, 60, 100).rename(columns={'Strat Returns':'All_OOS_Returns'})

In [108]:
oos_prediction_sum = pd.concat([OOS_DP_predict.T, OOS_EP_predict.T, OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T

In [109]:
def calculate_strategy_performance(OOS_DP_predict, OOS_EP_predict, OOS_all_predict, OOS_EPDP_predict, gmo_excess_ret, rf):

    strats = [('DP', OOS_DP_predict.dropna()),
              ('EP', OOS_EP_predict.dropna()),
              ('DP-EP', OOS_EPDP_predict.dropna()),
              ('All', OOS_all_predict.dropna()),
              ('SPY', gmo_excess_ret.loc[OOS_all_predict.index[0]:, ['SPY']].rename(columns={'SPY': 'SPY_OOS_Returns'})),
              ('US3M', rf['US3M'].to_frame('US3M_OOS_Returns'))]

    factor = gmo_excess_ret.loc[:, ['SPY']]
    strat_summary = []

    for k, strat in strats:
        perf_summary = performance_summary(strat)
        perf_summary['Negative Risk Premium Months'] = len(strat[strat[k + '_OOS_Returns'] - rf['US3M'] < 0])
        perf_summary['Total Months'] = len(strat)
        perf_summary.index = [k]
        reg = regression_based_performance(factor[strat.index[0]:], strat, 0)
        perf_summary['Market Beta'] = reg[0][0]
        perf_summary['Market Alpha'] = reg[3]
        perf_summary['Market Information Ratio'] = reg[2]
        strat_summary.append(perf_summary)

    strat_summary_df = pd.concat(strat_summary)
    return strat_summary_df.loc[:, ['Mean', 'Volatility', 'Sharpe Ratio', 'VaR (0.05)', 'Max Drawdown', 'Market Beta', 'Market Alpha', 'Market Information Ratio']]


In [110]:
strat_summary_df = calculate_strategy_performance(OOS_DP_predict, OOS_EP_predict, OOS_all_predict, OOS_EPDP_predict, gmo_excess_ret, rf)
strat_summary_df

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,0.079626,0.173732,0.458326,-0.071179,-0.551925,0.994490,0.012756,0.163440
EP,0.082373,0.163741,0.503066,-0.068431,-0.583693,0.549660,0.045420,0.325618
DP-EP,0.096815,0.226111,0.428174,-0.071698,-0.760910,0.469532,0.065244,0.305011
All,0.112022,0.247893,0.451897,-0.071882,-0.804959,0.490154,0.079068,0.335312
SPY,0.067239,0.156070,0.430826,-0.080066,-0.560012,1.000000,0.000000,6.067027
US3M,0.023803,0.006165,3.860934,0.000020,0.000000,-0.001179,0.023892,3.877706


### <br><br> How much better/worse is the OOS Earnings-Price ratio strategy compared to the in-sample version of 3.2?

Answer: compared to in-sample forecasts, OOS Earnings-Price ratio strategy shows lower mean returns and higher volatility, that's why Sharpe ratio becomes almost two times less. Also, tails risks metrics are worse as well. 

### 4.3) Re-do problem 3.3 using this OOS forecast. <br><br> Is the point-in-time version of the strategy riskier?

In [111]:
def summarize_strategies(strategies, risk_free_rate, excess_returns_factor, start_date, end_date):
    strat_summary = pd.DataFrame()

    for strat_name, strat_data in strategies:

        strat_data = strat_data[start_date:end_date]

        strat_return_column = strat_data.columns[0]  
        perf_summary = performance_summary(strat_data)

        excess_returns = strat_data[strat_return_column] - risk_free_rate[start_date:end_date]['US3M']
        perf_summary['Negative Risk Premium Months'] = (excess_returns < 0).sum()
        perf_summary['Total Months'] = len(strat_data)
        perf_summary.index = [strat_name]

        aligned_factor_data = excess_returns_factor.loc[strat_data.index]
        aligned_strat_data = strat_data[strat_return_column]

        reg = regression_based_performance(aligned_factor_data, aligned_strat_data, 0)

        perf_summary['Market Beta'] = reg[0][0]
        perf_summary['Market Alpha'] = reg[3]
        perf_summary['Market Information Ratio'] = reg[2]

        strat_summary = pd.concat([strat_summary, perf_summary])

    return strat_summary


In [112]:
oos_prediction_sum = pd.concat([OOS_DP_predict.T,OOS_EP_predict.T,OOS_all_predict.T])
oos_prediction_sum = oos_prediction_sum.T

strategies = [('DP', OOS_DP_predict.dropna()),
              ('EP', OOS_EP_predict.dropna()),
              ('DP-EP-US10Y', fac3_strat_rtn.dropna()),
              ('Risk Free Rate', rf['US3M'].to_frame('Forecasted Return'))
             ]
factor = gmo_excess_ret.loc[:,['SPY']]
start_date = '2000'
end_date = '2011'

strat_summary_df = summarize_strategies(strategies, rf[['US3M']], factor, start_date, end_date)
strat_summary_df.loc[:, ['Mean', 'Volatility', 'Sharpe Ratio', 'VaR (0.05)', 'Max Drawdown', 'Market Beta', 'Market Alpha', 'Market Information Ratio']]

,Mean,Volatility,Sharpe Ratio,VaR (0.05),Max Drawdown,Market Beta,Market Alpha,Market Information Ratio
DP,-0.010895,0.163221,-0.066749,-0.094730,-0.551925,0.952248,-0.006228,-0.124928
EP,0.038768,0.195919,0.197877,-0.085329,-0.583693,0.296065,0.040224,0.211832
DP-EP-US10Y,0.061471,0.158851,0.386972,-0.077629,-0.524606,0.736802,0.065088,0.627126
Risk Free Rate,0.023062,0.005785,3.986632,0.000035,0.000000,-0.002853,0.023052,3.997186


In [113]:
neg_risk_premium = strat_summary_df.loc[:,['Negative Risk Premium Months','Total Months']]
neg_risk_premium['Negative Risk Premium Months (%)'] = round(neg_risk_premium['Negative Risk Premium Months'] *100/ neg_risk_premium['Total Months'], 2)
neg_risk_premium

,Negative Risk Premium Months,Total Months,Negative Risk Premium Months (%)
DP,69,144,47.92
EP,66,144,45.83
DP-EP-US10Y,62,144,43.06
Risk Free Rate,0,144,0.00


**Answer**: strategy takes extra risk because volatility increases, mean return decreases, and hence Sharpe ratio decreases. In addition, tail risk metrics becomes worse.